In [1]:
import bs4
from bs4 import BeautifulSoup
import os
from os.path import join, splitext

from tqdm import tqdm # remove on DEV

In [2]:
files_dir = '../examples'
output_dir = '../output'
os.makedirs(output_dir, exist_ok=True)

In [3]:
# CONSTANTS
max_mag = 40
ast_mag_const = 1
ast_1_const = 10
ast_2_const = 10
max_arcsec = 10

In [4]:
def is_asteroid(pre_text):
    if 'The object was found in astcheck' in pre_text:
        pre_text_split = pre_text.split('\n')
        
        obj_mag = float(pre_text_split[1].strip().split()[4])
        assert obj_mag < max_mag
        
        for idx, el in enumerate(pre_text_split):
            if 'astcheck' in el:
                ast_idx = idx - 1
                break

        ast_split = pre_text_split[ast_idx].strip().split()
        ast_1 = float(ast_split[2])
        ast_2 = float(ast_split[3])
        ast_mag = float(ast_split[5])
        assert ast_mag < max_mag
        
        return (abs(obj_mag - ast_mag) <= ast_mag_const and 
                ast_1 <= ast_1_const and
                ast_2 <= ast_2_const)
    else:
        return False

def is_variable_star(pre_text):
    if 'The object was found in VSX' in pre_text:
        pre_text_split = pre_text.split('\n')
        for idx, el in enumerate(pre_text_split):
            if 'VSX' in el:
                vs_idx = idx + 1
                break
        vs_arcsec = int(pre_text_split[vs_idx].strip().split()[0][:-1])
        return vs_arcsec <= max_arcsec
    else:
        return False

def is_ast_or_vs(pre_text):
    return is_asteroid(pre_text) or is_variable_star(pre_text)

In [5]:
for file_path in tqdm(os.listdir(files_dir)):
    with open(join(files_dir, file_path), 'r') as f:
        content = f.read()
    
    a_name_first_occurance = content.find('<a name')
    head = content[: a_name_first_occurance]
    transients = content[a_name_first_occurance:].split('<hr>')[:-1]
    
    ast_or_vs_s = []
    for transient in tqdm(transients):
        ast_or_vs_s.append(is_ast_or_vs(BeautifulSoup(transient).pre.text))
    
    not_ast_and_not_vs = []
    for transient, ast_or_vs_ in zip(transients, ast_or_vs_s):
        if not ast_or_vs_:
            not_ast_and_not_vs.append(transient)

    output = head + '<hr>'.join(not_ast_and_not_vs) + '\n<hr></body></html>'
    with open(join(output_dir, splitext(file_path)[0]+'___'+splitext(file_path)[1]), 'w') as f:
        f.write(output)

100%|██████████| 4/4 [00:00<00:00, 263.55it/s]

 19%|█▉        | 3/16 [00:00<00:06,  1.97it/s]
0it [00:00, ?it/s]

100%|██████████| 21/21 [00:00<00:00, 271.26it/s]

100%|██████████| 1/1 [00:00<00:00, 258.70it/s]

100%|██████████| 18/18 [00:00<00:00, 276.52it/s]

 75%|███████▌  | 12/16 [00:03<00:00,  4.25it/s]
0it [00:00, ?it/s]

100%|██████████| 19/19 [00:00<00:00, 267.74it/s]

100%|██████████| 16/16 [00:04<00:00,  3.31it/s]
